### What if we identified using pinyin homophones AND meanings

In [31]:
import pandas as pd
import jieba
from pypinyin import lazy_pinyin
from difflib import SequenceMatcher

In [32]:
comments.head()

,index,weibo_id,comment_text,gender,location,like,date,label,category,target
0,0,J1zF7smOR,她们不结婚不要孩子，却觉得我们这种结婚有孩子的是婚驴,女,广东,14,12/05/2020 15:04,1,SCB,G
1,1,GvXf0ymdJ,妈的什么时候有人来性侵我一下啊 不要封口费的那种,男,其他,36,23/08/2018 13:10,1,SO,I
2,2,J2XeR8rPn,我觉得这就是对一类人所有的特点进行形容吧，“爹味”也是贬义词，为什么不觉得有性别歧视意味呢？,女,浙江,0,21/05/2020 16:12,1,MA,G
3,3,IxpfTCMAo,姐姐你不要急，关注久的都知道她骂的一切归根结底都是渴婚的，她并不骂学艺术的，她骂的是不肯像培养儿子一样培养女儿，给女儿学跳舞弹琴之类想给女儿加码卖更好价钱的，她并不是骂艺术，而是目的性的学，并且是恶心目的，如果没有这个想法有啥好气的啊？而且明明骂男人更毒更厉害，都涉及人身攻击了呀,女,江苏,1,31/03/2020 13:57,1,MA,I
4,4,J4B0Mps5Q,"给他们争取权利的不是女权是腐女，你该不会说腐女都女权吧？连女同性恋都不是全部女权，腐女？,男同性恋好吗？女同性恋在你眼里又隐身了",女,其他,0,##########,1,MA,G


In [33]:
flag_terms = {
    "misogyny": ["女拳", "田园女权", "工具人", "婚驴", "世界公交车", "婊子", "爹味", "国男"],
    "political": ["红色", "毛主席", "资本主义", "公正", "打拳", "审判", "社会"]
}

#finding pinyin similarity
def is_pinyin_similar(word1, word2):
    p1 = lazy_pinyin(word1)
    p2 = lazy_pinyin(word2)
    return SequenceMatcher(None, ''.join(p1), ''.join(p2)).ratio() > 0.9

results = []
for _, row in comments.iterrows():
    text = row["comment_text"]
    tokens = list(jieba.cut(text))
    matched_terms = []

    for token in tokens:
        for category, terms in flag_terms.items():
            for term in terms:
                if token == term or is_pinyin_similar(token, term):
                    matched_terms.append((token, category, term))

    results.append({
        "original_text": text,
        "flagged_terms": [m[0] for m in matched_terms],
        "match_basis": [f"{m[0]} ~ {m[2]}" for m in matched_terms]
    })

# STEP 5: Output as DataFrame
flagged_df = pd.DataFrame(results)
flagged_df

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
DEBUG:jieba:Loading model from cache /tmp/jieba.cache
Loading model cost 0.800 seconds.
DEBUG:jieba:Loading model cost 0.800 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


,original_text,flagged_terms,match_basis
0,她们不结婚不要孩子，却觉得我们这种结婚有孩子的是婚驴,[],[]
1,妈的什么时候有人来性侵我一下啊 不要封口费的那种,[],[]
2,我觉得这就是对一类人所有的特点进行形容吧，“爹味”也是贬义词，为什么不觉得有性别歧视意味呢？,[爹味],[爹味 ~ 爹味]
3,姐姐你不要急，关注久的都知道她骂的一切归根结底都是渴婚的，她并不骂学艺术的，她骂的是不肯像培养儿子一样培养女儿，给女儿学跳舞弹琴之类想给女儿加码卖更好价钱的，她并不是骂艺术，而是目的性的学，并且是恶心目的，如果没有这个想法有啥好气的啊？而且明明骂男人更毒更厉害，都涉及人身攻击了呀,[],[]
4,"给他们争取权利的不是女权是腐女，你该不会说腐女都女权吧？连女同性恋都不是全部女权，腐女？,男同性恋好吗？女同性恋在你眼里又隐身了","[女权, 女权, 女权]","[女权 ~ 女拳, 女权 ~ 女拳, 女权 ~ 女拳]"
...,...,...,...
8964,之前一直是五个男生那种，你们批评我吧,[],[]
8965,中国的小孩也可以随母亲姓啊 我一个朋友就是随母亲姓的,[],[]
8966,所有为韦涛卖命而去攻击别人的炮灰都快点死.......咒语灵灵灵。。。。。。,[],[]
8967,本质是阶级矛盾……比如你是个资本家，别人抱你大腿的时候就不会管你是男还是女,[],[]


In [34]:
 flagged_df['flagged_terms'].tolist()

[[],
 [],
 ['爹味'],
 [],
 ['女权', '女权', '女权'],
 ['女拳'],
 [],
 [],
 [],
 [],
 [],
 ['社会', '社会', '社会'],
 [],
 ['社会', '社会', '地位'],
 ['女权', '女权'],
 [],
 [],
 ['女权', '女权', '地位'],
 [],
 [],
 [],
 [],
 [],
 ['女权', '女权'],
 [],
 [],
 ['社会', '地位'],
 [],
 ['社会', '社会', '社会', '社会', '地位', '地位'],
 [],
 ['女权', '女权'],
 ['社会'],
 [],
 ['女权', '女权', '女权', '女权'],
 ['女权', '女权', '过年'],
 [],
 [],
 [],
 ['女权'],
 [],
 ['社会', '地位'],
 [],
 [],
 ['女权'],
 ['社会'],
 [],
 [],
 [],
 ['社会', '女权'],
 ['女权'],
 ['地位'],
 [],
 [],
 [],
 [],
 ['红色'],
 ['女权', '女权'],
 [],
 [],
 [],
 [],
 ['女权'],
 [],
 [],
 ['社会'],
 [],
 ['女权', '女权'],
 [],
 [],
 [],
 [],
 ['社会'],
 [],
 [],
 ['女权'],
 ['地位', '地位', '地位', '社会'],
 [],
 [],
 ['公正'],
 [],
 ['社会'],
 ['社会', '地位', '女权', '女权'],
 [],
 ['打拳'],
 ['女权'],
 ['女权'],
 ['国腩'],
 [],
 [],
 [],
 ['女拳'],
 ['女权'],
 [],
 [],
 ['女权'],
 ['社会', '女权'],
 [],
 [],
 ['女权', '女权'],
 [],
 ['女权', '女权'],
 ['深谙', '社会', '社会', '社会'],
 [],
 [],
 [],
 ['女权'],
 ['社会'],
 ['女权', '社会', '女权', '女拳', '女拳', '社会', '地位'],
 [],
 ['女权']